In [ ]:
import sys
# sys.path.append("/Users/pangtao/PycharmProjects/manipulation_russ")
sys.path.append("/home/amazon/PycharmProjects/manipulation_russ")

In [ ]:
# Imports
import os

import numpy as np

from pydrake.all import (
    AddMultibodyPlantSceneGraph, DiagramBuilder, RigidTransform,
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator, MeshcatVisualizerCpp,
    Role, MeshcatVisualizerParams, StartMeshcat, Box, Rgba)
from manipulation import running_as_notebook
from manipulation.meshcat_cpp_utils import MeshcatJointSliders

from qsim.model_paths import models_dir
from qsim.parser import QuasistaticParser
from qsim.simulator import QuasistaticSimulator

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

In [ ]:
# Build bimanual plant with object.
q_model_path = os.path.join(models_dir, 'q_sys', 'iiwa_planar_bimanual_box.yml')
q_parser = QuasistaticParser(q_model_path)

builder = DiagramBuilder()
plant, scene_graph, robot_models, object_models = \
    QuasistaticSimulator.create_plant_with_robots_and_objects(
        builder=builder,
        model_directive_path=q_parser.model_directive_path,
        robot_names=[name for name in q_parser.robot_stiffness_dict.keys()],
        object_sdf_paths={},
        time_step=1e-3,  # Only useful for MBP simulations.
        gravity=q_parser.q_sim_params.gravity)

In [ ]:
meshcat.Delete()
# Add box.
X_WB = RigidTransform([0.55, 0, 0.203])
meshcat.SetObject("box", Box(0.460, 0.457, 0.406), Rgba(0.5, 0.5, 0.5, 0.6))
meshcat.SetTransform("box", X_WB)


mvp = MeshcatVisualizerParams()
mvp.role = Role.kIllustration
visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat, mvp)
diagram = builder.Build()
context = diagram.CreateDefaultContext()

sliders = MeshcatJointSliders(meshcat, plant, context)
sliders.Run(visualizer, context)  #
